In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()

google_api_key = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)
response = llm.invoke("What are some of the pros and cons of Python as a programming language?")
print(response)


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

gemini_model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.7,
    max_tokens=4000,
)


class RecommendedTopic(BaseModel):
    topic: str = Field(description="The name of the recommended topic")
    description: str = Field(description="A brief description of why this topic is relevant")
    resource_url: str = Field(description="A relevant resource URL for this topic")

class TopicRecommendations(BaseModel):
    recommendations: List[RecommendedTopic] = Field(description="List of recommended related topics")
    
# Create prompt templates
report_prompt = ChatPromptTemplate.from_template(
    """
    You are an AI research assistant. Create a comprehensive, detailed report on the following topic:
    
    Topic: {topic}
    
    Your report should include:
    1. Introduction to the topic
    2. Key concepts and definitions
    3. Historical context and development
    4. Current state and applications
    5. Future directions and potential developments
    6. Conclusion
    
    Format your report with clear headings and subheadings. Make sure to provide in-depth analysis.
    """
)

recommendation_prompt = ChatPromptTemplate.from_template(
    """
    Based on the topic: {topic}
    
    Generate 5 relevant related topics that the user might be interested in researching next.
    For each recommendation, provide:
    1. The topic name
    2. A brief 1-2 sentence description of why it's relevant
    3. A relevant resource URL that would contain valuable information about this topic
    
    Your response must be formatted as a valid JSON object that matches this structure:
    {
        "recommendations": [
            {
                "topic": "Topic Name",
                "description": "Brief description of relevance",
                "resource_url": "https://example.com/relevant-page"
            },
            ...
        ]
    }
    
    Use reputable sources for your resource URLs. While you can't verify if the exact URLs exist,
    make them realistic and likely to contain quality information.
    """
)

report_chain = (
    {"topic": RunnablePassthrough()}
    | report_prompt
    | gemini_model
    | StrOutputParser()
)

recommendation_chain = (
    {"topic": RunnablePassthrough()}
    | recommendation_prompt
    | gemini_model
    | JsonOutputParser(pydantic_object=TopicRecommendations)
)

class ResearchAssistantAgent:
    def __init__(self):
        self.report_chain = report_chain
        self.recommendation_chain = recommendation_chain
    
    def generate_report(self, topic):
        """Generate a detailed report on the given topic"""
        return self.report_chain.invoke(topic)
    
    def generate_recommendations(self, topic):
        """Generate relevant topic recommendations using Gemini API"""
        try:
            recommendations_data = self.recommendation_chain.invoke(topic)
            
            formatted_recommendations = "# Related Topics You May Be Interested In\n\n"
            for i, rec in enumerate(recommendations_data.recommendations, 1):
                formatted_recommendations += f"## {i}. {rec.topic}\n"
                formatted_recommendations += f"{rec.description}\n"
                formatted_recommendations += f"[Learn more]({rec.resource_url})\n\n"
                
            return formatted_recommendations
        except Exception as e:
            backup_prompt = ChatPromptTemplate.from_template(
                """
                Based on the topic: {topic}
                
                Provide 5 relevant related topics that the user might be interested in researching next.
                For each recommendation, provide:
                1. The topic name
                2. A brief description of why it's relevant
                3. A relevant resource link
                
                Format your response as a markdown list.
                """
            )
            backup_chain = backup_prompt | gemini_model | StrOutputParser()
            return backup_chain.invoke({"topic": topic})
    
    def run(self):
        """Main agent loop"""
        print("🔍 AI Research Assistant Agent 🔍")
        print("--------------------------------")
        print("I can help you research any topic in depth.")
        
        while True:
            topic = input("\nWhat topic would you like to research? (or type 'exit' to quit): ")
            
            if topic.lower() == 'exit':
                print("Thank you for using the AI Research Assistant. Goodbye!")
                break
            
            if not topic.strip():
                print("Please enter a valid topic.")
                continue
            
            print(f"\nResearching '{topic}'... This may take a moment.")
            
            try:
                report = self.generate_report(topic)
                
                recommendations = self.generate_recommendations(topic)
                
                print("\n" + "="*50)
                print(f"RESEARCH REPORT: {topic.upper()}")
                print("="*50 + "\n")
                print(report)
                
                print("\n" + "="*50)
                print("RECOMMENDED RELATED TOPICS")
                print("="*50 + "\n")
                print(recommendations)
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    agent = ResearchAssistantAgent()
    agent.run()